In [1]:
import datetime as dt

import numpy as np
import pandas as pd
import yfinance as yf

pd.options.display.float_format = "{:,.2f}".format

In [2]:
class Proj_1_Class:
    def __init__(self, assets, benchmarks, adj_close):
        self.assets = assets
        self.benchmarks = benchmarks
        self.adj_close = adj_close

        self.returns = self.adj_close.pct_change()

    def asset_comparison(self):
        self.output_table_1 = pd.DataFrame(index=self.assets)

        # Weight Calculation
        self.output_table_1["Weight"] = 1 / len(self.assets)

        # Annualized Volatility Calculation
        self.output_table_1["Annualized Volatility"] = (
            self.returns[-(21 * 3) :].std() ** (1 / np.sqrt(4)) * 100
        )

        # Beta Calculations
        self.beta = self.returns[-252:].cov() / self.returns[-252:].var()
        for bench in self.benchmarks:
            self.output_table_1[bench + " Beta"] = self.beta[bench]

        # Calculate Drawdowns
        self.weekly_dd = (
            self.adj_close[-252:].rolling(5).max()
            - self.adj_close[-252:].rolling(5).min()
        )

        self.output_table_1["Avg Weekly Drawdown"] = self.weekly_dd.mean()
        self.output_table_1["Max Weekly Drawdown"] = self.weekly_dd.max()

        # 10 Yr Total Return
        self.output_table_1["10yr Return"] = (
            self.adj_close.pct_change(len(self.adj_close) - 1)[-1:].T * 100
        )

        # 10 Yr Annualized Total Return
        self.output_table_1["Ann 10yr Return"] = self.output_table_1["10yr Return"] ** (
            1 / np.sqrt(252)
        )

        # Output resulting table, transposing for readability
        return self.output_table_1.T

    def portfolio_comparison(self):
        # Create output table
        self.output_table_2 = pd.DataFrame(index=self.benchmarks)

        # Create equal weighted portfolio columns
        self.adj_close["ewp"] = self.adj_close[self.assets].mean(axis=1)
        self.returns["ewp"] = self.returns[self.assets].mean(axis=1)

        # Calculate correlation to EWP
        self.output_table_2["Corr"] = self.returns[-252:].corr()["ewp"]

        # Calculate covariance between ETF and EWP
        self.output_table_2["Cov_2"] = (self.returns[-252:] * 100).cov()["ewp"]

        # Calculate Tracking Error
        self.output_table_2["Tracking Error"] = 0
        for bench in self.benchmarks:
            self.output_table_2.loc[bench, "Tracking Error"] = (
                self.returns[bench] - self.returns["ewp"]
            ).std() * 100

        return self.output_table_2.T

In [8]:
# Set variables for downloading data
assets = ["TSLA", "AAPL", "MSFT", "NVDA", "AMZN", "AMD", "GME"]
benchmarks = ["SPY", "IWM", "DIA"]
end_date = dt.datetime.today()
start_date = end_date - dt.timedelta(10 * 365)

# Download data & calculate returns
adj_close = yf.download(assets + benchmarks, start=start_date, end=end_date)[
    "Adj Close"
]

[*********************100%***********************]  10 of 10 completed


In [9]:
P1C = Proj_1_Class(assets=assets, benchmarks=benchmarks, adj_close=adj_close)

In [10]:
P1C.asset_comparison()

,TSLA,AAPL,MSFT,NVDA,AMZN,AMD,GME
Weight,0.14,0.14,0.14,0.14,0.14,0.14,0.14
Annualized Volatility,19.61,16.05,16.00,19.40,17.59,19.71,21.90
SPY Beta,1.75,1.29,1.28,2.20,1.61,2.05,1.94
IWM Beta,1.54,1.00,0.96,1.86,1.30,1.75,1.98
DIA Beta,1.79,1.48,1.44,2.40,1.74,2.26,2.07
Avg Weekly Drawdown,24.03,7.32,12.76,16.92,8.66,8.15,3.88
Max Weekly Drawdown,64.67,16.83,30.09,51.19,22.57,20.16,16.61
10yr Return,"6,527.79",743.29,981.05,"5,710.63",583.95,"2,425.48",365.67
Ann 10yr Return,1.74,1.52,1.54,1.72,1.49,1.63,1.45


In [ ]:
# credit to professor John Droescher's code